In [ ]:
from zipfile import ZipFile
file_name = ('data.zip')
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("Completed")

Completed


In [ ]:
import os
data_directory = os.path.join("data")
os.listdir(data_directory)

['Train_Data.csv', 'Test_Data.csv', 'Submit.csv']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding
#from tensorflow.keras import layers
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
dataset_train = pd.read_csv('data/Train_Data.csv')

In [ ]:
dataset_train.head()

,Title,Date,Abstract,Subjects
0,The Neutrino Ball Model of a Quasar,1993-04-07,It is suggested that the nonorthodox model o...,['Astrophysics']
1,Charged Boson Stars and Vacuum Instabilities,1993-05-12,We consider charged boson stars and study th...,"['Astrophysics', 'General Relativity and Quant..."
2,Pre-main sequence lithium burning. I. Weak T T...,1993-08-31,We derive lithium abundances in 53 T Tauri s...,['Astrophysics']
3,Very High Energy Gamma-Rays from AGN: Cascadin...,1993-12-18,Recent high energy gamma-ray observations (E...,['Astrophysics']
4,From the Owl to the Eskimo: the Radiation-Gasd...,1994-04-29,We present the results of two-dimensional ra...,['Astrophysics']


In [ ]:
dataset_train["Date"] = pd.to_datetime(dataset_train["Date"])

dataset_train = dataset_train.sort_values(by="Date")
dataset_train.head()

,Title,Date,Abstract,Subjects
0,The Neutrino Ball Model of a Quasar,1993-04-07,It is suggested that the nonorthodox model o...,['Astrophysics']
1,Charged Boson Stars and Vacuum Instabilities,1993-05-12,We consider charged boson stars and study th...,"['Astrophysics', 'General Relativity and Quant..."
2,Pre-main sequence lithium burning. I. Weak T T...,1993-08-31,We derive lithium abundances in 53 T Tauri s...,['Astrophysics']
3,Very High Energy Gamma-Rays from AGN: Cascadin...,1993-12-18,Recent high energy gamma-ray observations (E...,['Astrophysics']
4,From the Owl to the Eskimo: the Radiation-Gasd...,1994-04-29,We present the results of two-dimensional ra...,['Astrophysics']


In [ ]:
col = ['Title', 'Date','Subjects']
df = dataset_train[col]

In [ ]:
df.head()

,Title,Date,Subjects
0,The Neutrino Ball Model of a Quasar,1993-04-07,['Astrophysics']
1,Charged Boson Stars and Vacuum Instabilities,1993-05-12,"['Astrophysics', 'General Relativity and Quant..."
2,Pre-main sequence lithium burning. I. Weak T T...,1993-08-31,['Astrophysics']
3,Very High Energy Gamma-Rays from AGN: Cascadin...,1993-12-18,['Astrophysics']
4,From the Owl to the Eskimo: the Radiation-Gasd...,1994-04-29,['Astrophysics']


In [ ]:
df['Subjects'].value_counts()

['Astrophysics - Astrophysics of Galaxies']                                                                                                                                                              4408
['Astrophysics - High Energy Astrophysical Phenomena']                                                                                                                                                   3453
['Astrophysics - Solar and Stellar Astrophysics']                                                                                                                                                        2874
['Astrophysics - Earth and Planetary Astrophysics']                                                                                                                                                      2144
['Astrophysics - Cosmology and Nongalactic Astrophysics']                                                                                                                       

In [ ]:
possible_labels = df.Subjects.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{"['Astrophysics']": 0,
 "['Astrophysics', 'General Relativity and Quantum Cosmology']": 1,
 "['High Energy Physics - Phenomenology', 'Astrophysics', 'High Energy Physics - Theory']": 2,
 "['General Relativity and Quantum Cosmology', 'Astrophysics', 'High Energy Physics - Theory']": 3,
 "['High Energy Physics - Phenomenology', 'Astrophysics']": 4,
 "['High Energy Physics - Phenomenology', 'Astrophysics', 'Nuclear Theory']": 5,
 "['Astrophysics', 'Condensed Matter']": 6,
 "['High Energy Physics - Experiment', 'Astrophysics', 'High Energy Physics - Phenomenology']": 7,
 "['Condensed Matter - Soft Condensed Matter', 'Astrophysics', 'Condensed Matter - Mesoscale and Nanoscale Physics', 'High Energy Physics - Theory']": 8,
 "['Astrophysics', 'High Energy Physics - Experiment', 'High Energy Physics - Phenomenology']": 9,
 "['Astrophysics', 'General Relativity and Quantum Cosmology', 'Nonlinear Sciences - Chaotic Dynamics']": 10,
 "['Condensed Matter - Statistical Mechanics', 'Astrophysics', 

In [ ]:
df['label'] = df.Subjects.replace(label_dict)

In [ ]:
df.head()

,Title,Date,Subjects,label
0,The Neutrino Ball Model of a Quasar,1993-04-07,['Astrophysics'],0
1,Charged Boson Stars and Vacuum Instabilities,1993-05-12,"['Astrophysics', 'General Relativity and Quant...",1
2,Pre-main sequence lithium burning. I. Weak T T...,1993-08-31,['Astrophysics'],0
3,Very High Energy Gamma-Rays from AGN: Cascadin...,1993-12-18,['Astrophysics'],0
4,From the Owl to the Eskimo: the Radiation-Gasd...,1994-04-29,['Astrophysics'],0


In [ ]:
df

,Title,Date,Subjects,label
0,The Neutrino Ball Model of a Quasar,1993-04-07,['Astrophysics'],0
1,Charged Boson Stars and Vacuum Instabilities,1993-05-12,"['Astrophysics', 'General Relativity and Quant...",1
2,Pre-main sequence lithium burning. I. Weak T T...,1993-08-31,['Astrophysics'],0
3,Very High Energy Gamma-Rays from AGN: Cascadin...,1993-12-18,['Astrophysics'],0
4,From the Owl to the Eskimo: the Radiation-Gasd...,1994-04-29,['Astrophysics'],0
...,...,...,...,...
29160,Kinematics of CIV and [OIII] emission in lumin...,2019-04-30,['Astrophysics - Astrophysics of Galaxies'],43
29161,Cherenkov-Plenoscope,2019-04-30,['Astrophysics - Instrumentation and Methods f...,311
29162,Cosmology with dropout selection: Straw-man su...,2019-04-30,['Astrophysics - Cosmology and Nongalactic Ast...,44
29164,Space-time geometry of spiral galaxy halo,2019-04-30,"['General Relativity and Quantum Cosmology', '...",415


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  )
X_train = np.asarray(X_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['Subjects', 'label', 'data_type']).count()

Title  \
Subjects                                           label data_type          
['Astrophysics - Astrophysics of Galaxies', 'As... 1452  train          1   
['Astrophysics - Astrophysics of Galaxies', 'As... 474   train          1   
                                                         val            1   
['Astrophysics - Astrophysics of Galaxies', 'As... 423   train          2   
['Astrophysics - Astrophysics of Galaxies', 'As... 1370  train          1   
...                                                                   ...   
['Statistics - Methodology', 'Astrophysics - In... 417   train          1   
['Statistics - Methodology', 'Astrophysics - In... 816   train          1   
['Statistics - Methodology', 'Astrophysics - In... 393   val            1   
['Statistics - Methodology', 'Astrophysics - In... 594   train          2   
['Statistics - Other Statistics', 'Astrophysics... 1441  train          1   

                                                                    Date  
Subjects                                           label data_type        
['Astrophysics - Astrophysics of Galaxies', 'As... 1452  train         1  
['Astrophysics - Astrophysics of Galaxies', 'As... 474   train         1  
                                                         val           1  
['Astrophysics - Astrophysics of Galaxies', 'As... 423   train         2  
['Astrophysics - Astrophysics of Galaxies', 'As... 1370  train         1  
...                                                                  ...  
['Statistics - Methodology', 'Astrophysics - In... 417   train         1  
['Statistics - Methodology', 'Astrophysics - In... 816   train         1  
['Statistics - Methodology', 'Astrophysics - In... 393   val           1  
['Statistics - Methodology', 'Astrophysics - In... 594   train         2  
['Statistics - Other Statistics', 'Astrophysics... 1441  train         1  

[1760 rows x 2 columns]

In [ ]:
df

,Title,Date,Subjects,label,data_type
0,The Neutrino Ball Model of a Quasar,1993-04-07,['Astrophysics'],0,train
1,Charged Boson Stars and Vacuum Instabilities,1993-05-12,"['Astrophysics', 'General Relativity and Quant...",1,train
2,Pre-main sequence lithium burning. I. Weak T T...,1993-08-31,['Astrophysics'],0,train
3,Very High Energy Gamma-Rays from AGN: Cascadin...,1993-12-18,['Astrophysics'],0,train
4,From the Owl to the Eskimo: the Radiation-Gasd...,1994-04-29,['Astrophysics'],0,train
...,...,...,...,...,...
29160,Kinematics of CIV and [OIII] emission in lumin...,2019-04-30,['Astrophysics - Astrophysics of Galaxies'],43,train
29161,Cherenkov-Plenoscope,2019-04-30,['Astrophysics - Instrumentation and Methods f...,311,train
29162,Cosmology with dropout selection: Straw-man su...,2019-04-30,['Astrophysics - Cosmology and Nongalactic Ast...,44,train
29164,Space-time geometry of spiral galaxy halo,2019-04-30,"['General Relativity and Quantum Cosmology', '...",415,train


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29191 entries, 0 to 29190
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Title      29191 non-null  object        
 1   Date       29191 non-null  datetime64[ns]
 2   Subjects   29191 non-null  object        
 3   label      29191 non-null  int64         
 4   data_type  29191 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 2.3+ MB


In [ ]:
X = df.drop(["label","Subjects"],axis=1)
y = df["label"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.25, random_state=42)

In [ ]:
epochs = 10
emb_dim = 128
batch_size = 256

In [ ]:
print((X_train.shape, y_train.shape, X_test.shape, y_test.shape))

model = Sequential()
model.add(Embedding(8000, emb_dim, input_length=X.shape[1]))
model.add(Dropout(0.2))
model.add(LSTM(64, dropout=0.7, recurrent_dropout=0.7))
model.add(Dense(1467, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',patience=7, min_delta=0.0001)])

((21893, 3), (21893,), (7298, 3), (7298,))
Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 3, 128)            1024000   
                                                                 
 dropout_7 (Dropout)         (None, 3, 128)            0         
                                                                 
 lstm_10 (LSTM)              (None, 64)                49408     
                                                                 
 dense_9 (Dense)             (None, 1467)              95355     
                                                                 
Total params: 1,168,763
Trainable params: 1,168,763
Non-trainable params: 0
_________________________________________________________________
None


ValueError: ignored